In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm

from gensim.models import KeyedVectors

import multiprocessing as mp
from spacy.tokens import Doc
from typing import List

In [2]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/MyDrive/data/data/"

Mounted at /content/drive


In [ ]:
dmeeting = pd.read_parquet(data_path + 'data_meeting.parquet')
data_agenda1 = pd.read_parquet(data_path + 'data_agenda1.parquet')
data_agenda2 = pd.read_parquet(data_path + 'data_agenda2.parquet')
data_agenda3 = pd.read_parquet(data_path + 'data_agenda3.parquet')
data_speech1 = pd.read_parquet(data_path + 'data_speech1.parquet')
data_speech2 = pd.read_parquet(data_path + 'data_speech2.parquet')
data_speech3 = pd.read_parquet(data_path + 'data_speech3.parquet')
parMem = pd.read_parquet(data_path + 'parliament_members.parquet')

dagenda = pd.concat([data_agenda1, data_agenda2, data_agenda3], axis=0)
dspeech = pd.concat([data_speech1, data_speech2, data_speech3], axis=0)

annotation_data = pd.read_csv(data_path + 'annotation_data.csv')

In [ ]:
#dmeeting = pd.read_parquet('./data/data_meeting.parquet')
#data_agenda1 = pd.read_parquet('./data/data_agenda1.parquet')
#data_agenda2 = pd.read_parquet('./data/data_agenda2.parquet')
#data_agenda3 = pd.read_parquet('./data/data_agenda3.parquet')
#data_speech1 = pd.read_parquet('./data/data_speech1.parquet')
#data_speech2 = pd.read_parquet('./data/data_speech2.parquet')
#data_speech3 = pd.read_parquet('./data/data_speech3.parquet')
#parMem = pd.read_parquet('./data/parliament_members.parquet')

#dagenda = pd.concat([data_agenda1, data_agenda2, data_agenda3], axis=0)
#dspeech = pd.concat([data_speech1, data_speech2, data_speech3], axis=0)

#annotation_data = pd.read_csv('data/annotation_data.csv')

In [ ]:
agendaC = annotation_data[annotation_data['label'] == 'C']
speechC = dspeech[dspeech[['meeting_id', 'agenda_item_id']].apply(tuple, axis=1).isin(agendaC[['meeting_id', 'agenda_item_id']].apply(tuple, axis=1))]
speechC['label'] = 'C'
agendaNC = annotation_data[annotation_data['label'] == 'NC']
speechNC = dspeech[dspeech[['meeting_id', 'agenda_item_id']].apply(tuple, axis=1).isin(agendaNC[['meeting_id', 'agenda_item_id']].apply(tuple, axis=1))]
speechNC = pd.merge(speechNC, agendaNC[['meeting_id', 'agenda_item_id', 'group']], on=['meeting_id', 'agenda_item_id'], how='left')
speechNC['label'] = 'NC'

<ipython-input-6-7c36f430cb1f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speechC['label'] = 'C'


In [ ]:
!python3 -m spacy download 'da_core_news_sm'

2023-11-30 07:51:46.933650: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 07:51:46.933718: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 07:51:46.933749: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 07:51:48.190254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('da_core_news_sm')


### Tokenize Speech Text Items Fast

In [ ]:
nlp = spacy.load("da_core_news_sm")
exception_list = ['CO2', 'co2']

def preprocess_text(doc: Doc) -> List[str]:
    processed_tokens = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha or token.text in exception_list]
    return processed_tokens

def preprocess_text_lemma(doc: Doc) -> List[str]:
    processed_tokens_lemma = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha or token.text in exception_list]
    return processed_tokens_lemma

speech_comb = pd.concat([speechC, speechNC])
speech_comb = speech_comb.rename(columns={'label': 'label_agenda'})

# Use spaCy's pipe method to process text in batches
texts = speech_comb.speech_item_text.values.tolist()
docs = list(nlp.pipe(texts, n_process=16))  # Use multiprocessing

# Apply the text processing functions
speech_comb['processed_tokens'] = [preprocess_text(doc) for doc in docs]
speech_comb['processed_tokens_lemma'] = [preprocess_text_lemma(doc) for doc in docs]

In [ ]:
df = speech_comb[["meeting_id","agenda_item_id","speech_item_id","label_agenda","processed_tokens","processed_tokens_lemma"]]
df['id'] = df["meeting_id"].astype(str) + "_" + df["agenda_item_id"].astype(str) + "_" + df["speech_item_id"].astype(str)
df = df.drop(columns=["meeting_id","agenda_item_id","speech_item_id"])

<ipython-input-15-ec5fcbc98bbc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df["meeting_id"].astype(str) + "_" + df["agenda_item_id"].astype(str) + "_" + df["speech_item_id"].astype(str)


In [ ]:
df.to_csv(data_path + '/labeled_tokenized_data.csv', index=False)

In [3]:
df = pd.read_csv(data_path + 'labeled_tokenized_data.csv')
df["processed_tokens"] = df["processed_tokens"].apply(lambda x: eval(x))
df["processed_tokens_lemma"] = df["processed_tokens_lemma"].apply(lambda x: eval(x))

### Caluculate Odds


In [4]:
def caluculate_odds(freq1, freq2, word_count):
    if freq2 == 0:
        if freq1 == 0:
            return 0
        else:
            if freq1 > 0.0001:
              return 9999
            else:
              return 0
    else:
        if freq1 == 0:
            return 0
        else:
            return freq1/freq2

In [5]:
from collections import Counter

df_C = df[df['label_agenda']=='C']
df_NC = df[df['label_agenda']=='NC']

unique_words_C_list = [word for tokens in df_C.processed_tokens for word in tokens]
unique_words_NC_list = [word for tokens in df_NC.processed_tokens for word in tokens]

unique_words_lemma_C_list = [word for tokens in df_C.processed_tokens_lemma for word in tokens]
unique_words_lemma_NC_list = [word for tokens in df_NC.processed_tokens_lemma for word in tokens]

unique_words_C_dict = Counter(unique_words_C_list)
unique_words_NC_dict = Counter(unique_words_NC_list)

unique_words_lemma_C_dict = Counter(unique_words_lemma_C_list)
unique_words_lemma_NC_dict = Counter(unique_words_lemma_NC_list)

unique_words_C_df = pd.DataFrame.from_dict(unique_words_C_dict, orient='index', columns=['word_count_C']).reset_index().rename(columns={'index': 'word'})
unique_words_NC_df = pd.DataFrame.from_dict(unique_words_NC_dict, orient='index', columns=['word_count_NC']).reset_index().rename(columns={'index': 'word'})


unique_words_C_lemma_df = pd.DataFrame.from_dict(unique_words_lemma_C_dict, orient='index', columns=['word_count_C']).reset_index().rename(columns={'index': 'word'})
unique_words_NC_lemma_df = pd.DataFrame.from_dict(unique_words_lemma_NC_dict, orient='index', columns=['word_count_NC']).reset_index().rename(columns={'index': 'word'})


In [6]:
unique_words = pd.merge(unique_words_C_df, unique_words_NC_df, on='word', how='outer')
unique_words[['word_count_C', 'word_count_NC']] = unique_words[['word_count_C', 'word_count_NC']].fillna(0)
unique_words['frequency_C'] = unique_words['word_count_C'] / sum(unique_words_C_df['word_count_C'])
unique_words['frequency_NC'] = unique_words['word_count_NC'] / sum(unique_words_NC_df['word_count_NC'])
unique_words["odds_C"] = unique_words.apply(lambda row: caluculate_odds(row['frequency_C'], row['frequency_NC'], row['word_count_C']), axis=1)
unique_words["odds_NC"] = unique_words.apply(lambda row: caluculate_odds(row['frequency_NC'], row['frequency_C'], row['word_count_NC']), axis=1)

In [7]:
unique_words_lemma = pd.merge(unique_words_C_lemma_df, unique_words_NC_lemma_df, on='word', how='outer')
unique_words_lemma[['word_count_C', 'word_count_NC']] = unique_words_lemma[['word_count_C', 'word_count_NC']].fillna(0)
unique_words_lemma['frequency_C'] = unique_words_lemma['word_count_C'] / sum(unique_words_C_lemma_df['word_count_C'])
unique_words_lemma['frequency_NC'] = unique_words_lemma['word_count_NC'] / sum(unique_words_NC_lemma_df['word_count_NC'])
unique_words_lemma["odds_C"] = unique_words_lemma.apply(lambda row: caluculate_odds(row['frequency_C'], row['frequency_NC'], row['word_count_C']), axis=1)
unique_words_lemma["odds_NC"] = unique_words_lemma.apply(lambda row: caluculate_odds(row['frequency_NC'], row['frequency_C'], row['word_count_NC']), axis=1)

### Decide a Threshold for Words List

In [10]:
unique_words[(unique_words.word_count_NC<3)&(unique_words.frequency_C >= 0.0001)].describe()

,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC
count,33.000000,33.000000,33.000000,3.300000e+01,33.000000,33.000000
mean,33.151515,0.636364,0.000182,2.349066e-07,5372.617817,0.001355
std,16.806102,0.742386,0.000092,2.740434e-07,4845.872948,0.001749
min,19.000000,0.000000,0.000104,0.000000e+00,141.501397,0.000000
25%,21.000000,0.000000,0.000115,0.000000e+00,417.056748,0.000000
50%,26.000000,0.000000,0.000143,0.000000e+00,9999.000000,0.000000
75%,36.000000,1.000000,0.000198,3.691390e-07,9999.000000,0.002398
max,81.000000,2.000000,0.000445,7.382779e-07,9999.000000,0.007067


In [15]:
unique_words[(unique_words.word_count_NC>2) & (unique_words.odds_C < 20) & (unique_words.odds_C > 10) & (unique_words.frequency_C > 0.0001)][0:50]

,word,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC
87,klimaområdet,28.0,34.0,0.000154,0.000013,12.266375,0.081524
757,fødevarer,142.0,180.0,0.000781,0.000066,11.750408,0.085103
901,miljømæssigt,26.0,32.0,0.000143,0.000012,12.102093,0.082630
930,udbygningen,22.0,32.0,0.000121,0.000012,10.240233,0.097654
983,energiafgifter,23.0,21.0,0.000126,0.000008,16.313444,0.061299
1010,per,208.0,278.0,0.001144,0.000103,11.144374,0.089731
1020,energistyrelsen,21.0,23.0,0.000115,0.000008,13.599677,0.073531
1050,co2,96.0,110.0,0.000528,0.000041,12.999171,0.076928
1141,teknologier,54.0,56.0,0.000297,0.000021,14.362924,0.069624
1182,affald,63.0,87.0,0.000346,0.000032,10.785950,0.092713


In [12]:
unique_words[(unique_words.word_count_NC>2) & (unique_words.odds_C > 20) & (unique_words.frequency_C > 0.0001)][0:50]

,word,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC
29,udledninger,21.0,12.0,0.000115,0.000004,26.066047,0.038364
34,reduktionsmål,26.0,13.0,0.000143,0.000005,29.789768,0.033569
40,reduktioner,35.0,10.0,0.000192,0.000004,52.132093,0.019182
242,fodslæbende,23.0,9.0,0.000126,0.000003,38.064703,0.026271
274,energipolitik,72.0,19.0,0.000396,0.000007,56.443770,0.017717
373,klimapolitik,65.0,26.0,0.000357,0.000010,37.237210,0.026855
397,fossile,96.0,51.0,0.000528,0.000019,28.037428,0.035667
398,brændsler,75.0,21.0,0.000412,0.000008,53.196014,0.018798
400,kul,38.0,10.0,0.000209,0.000004,56.600559,0.017668
461,energi,379.0,254.0,0.002084,0.000094,22.225043,0.044994


In [14]:
unique_words[(unique_words.word_count_C<3)&(unique_words.frequency_NC >= 0.0001)][:50]

,word,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC
3282,integration,1.0,277.0,0.000005,0.000102,0.053772,18.596990
3374,dagpenge,1.0,510.0,0.000005,0.000188,0.029206,34.239945
4259,karsten,1.0,400.0,0.000005,0.000148,0.037237,26.854859
6319,kontakt,2.0,352.0,0.000011,0.000130,0.084630,11.816138
6697,køn,2.0,428.0,0.000011,0.000158,0.069602,14.367349
7283,voksne,1.0,365.0,0.000005,0.000135,0.040808,24.505058
7690,arktis,1.0,437.0,0.000005,0.000161,0.034084,29.338933
7782,fængsel,2.0,416.0,0.000011,0.000154,0.071610,13.964526
7860,forældre,2.0,1054.0,0.000011,0.000389,0.028264,35.381276
8213,straf,2.0,435.0,0.000011,0.000161,0.068482,14.602329


In [ ]:
C_words_list = [
    'reduktioner', 'fodslæbende', 'energipolitik', 'klimapolitik', 'brændsler', 'kul',
    'klimakommissionen', 'klimapolitikken', 'biomasse', 'vind', 'biobrændstoffer',
    'biodiversitet', 'energiområdet', 'dalgaard', 'planter', 'plast', 'biodiversiteten',
    'skov', 'brændstof', 'elbilerne', 'biobrændsel', 'energiforsyning', 'skove',
    'energiforbruget', 'energiministeren', 'klimaministeren', 'biogas', 'delmål',
    'energipolitikken', 'energiforbrug', 'energiaftalen', 'energiaftale', 'energibesparelser',
    'vindenergi', 'energieffektivitet', 'vindmøller', 'vindmølleindustrien', 'vindindustrien',
    'energien', 'energibesparende', 'miljøklagenævnet', 'naturfredningsforening', 'arter',
    'solceller', 'energiforliget', 'testcenter', 'naturområde', 'hede', 'klimaloven', 'klimalov',
    'klimarådet', 'klimarådets', 'miljøøkonomiske', 'plantebaserede', 'pesticider',
    'luftforurening', 'øresund', 'luftforureningen', 'sprøjtemidler', 'solcelleanlæg',
    'befordringsfradraget', 'overskudsvarme', 'elektrificering', 'plastik', 'fiskeredskaber',
    'urørt', 'stenrev', 'elektrificeringen', 'miljøzonerne', 'sandsugning', 'naturplan'
]

In [ ]:
unique_words[(unique_words.word_count_C!=0) & (unique_words.odds_NC > 10) & (unique_words.frequency_NC > 0.0001)].describe()

,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,2.100000,627.150000,0.000012,0.000232,0.052189,22.856394
std,2.039442,522.169004,0.000011,0.000193,0.022218,9.810279
min,1.000000,277.000000,0.000005,0.000102,0.019047,10.238415
25%,1.000000,352.000000,0.000005,0.000130,0.037880,14.845702
50%,2.000000,455.500000,0.000011,0.000168,0.045114,22.166448
75%,2.000000,676.000000,0.000011,0.000250,0.067366,26.401683
max,15.000000,3592.000000,0.000082,0.001326,0.097671,52.501249


In [ ]:
NC_words_list = [
    'integration', 'dagpenge', 'unge', 'politiet', 'rettigheder', 'kvinder', 'kontakt',
    'køn', 'krisen', 'almene', 'voksne', 'børnene', 'arktis', 'fængsel', 'sociale',
    'forældre', 'straf', 'grønland', 'aktindsigt', 'barn', 'regionerne', 'ligestilling',
    'redegørelsen', 'frivillige', 'arbejdsmarked', 'arbejdsmarkedet', 'su', 'personale',
    'elever', 'videregående', 'udlændinge', 'rusland', 'islamisk', 'forældrene', 'pension',
    'ledelse', 'medier', 'skole', 'mænd', 'vold', 'sygdom', 'reformer', 'grundloven',
    'løn', 'domstolene', 'arbejdsmiljø', 'psykisk', 'kontanthjælp', 'barnet', 'patienter',
    'overgreb', 'rigsfællesskabet', 'undervisning', 'politiets', 'finansielle', 'boligområder',
    'permanent'
]

In [ ]:
rare_C_words = unique_words[(unique_words.word_count_NC<3)&(unique_words.frequency_C >= 0.0001)].word.to_list()
rare_NC_words = unique_words[(unique_words.word_count_C<3)&(unique_words.frequency_NC >= 0.0001)].word.to_list()
C_words = unique_words[(unique_words.word_count_NC>2) & (unique_words.odds_C > 20) & (unique_words.frequency_C > 0.0001)].word.to_list()
NC_words = unique_words[(unique_words.word_count_C>2) & (unique_words.odds_NC > 10) & (unique_words.frequency_NC > 0.0001)].word.to_list()

In [ ]:
generic_words_list = unique_words[(unique_words.frequency_C>0.0015)&(unique_words.frequency_NC>0.0015)].word.to_list()

In [ ]:
import pickle

with open(data_path + 'generic_words_list.pkl', 'wb') as file:
    pickle.dump(generic_words_list, file)

with open(data_path + 'C_words_list.pkl', 'wb') as file:
    pickle.dump(C_words_list, file)

with open(data_path + 'NC_words_list.pkl', 'wb') as file:
    pickle.dump(NC_words_list, file)

In [ ]:
with open(data_path + 'rare_C_words.pkl', 'wb') as file:
    pickle.dump(rare_C_words, file)

with open(data_path + 'rare_NC_words.pkl', 'wb') as file:
    pickle.dump(rare_NC_words, file)

with open(data_path + 'C_words.pkl', 'wb') as file:
    pickle.dump(C_words, file)

with open(data_path + 'NC_words.pkl', 'wb') as file:
    pickle.dump(NC_words, file)

### Problem with Lemmas

In [ ]:
texts = unique_words.word.values.tolist()
docs = list(nlp.pipe(texts, n_process=16))

unique_words['word_lemma'] = [preprocess_text_lemma(doc) for doc in docs]
unique_words["word_lemma"] = unique_words.word_lemma.apply(lambda x: x[0])

In [ ]:
unique_words[unique_words.word=="stenrev"]

,word,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC,word_lemma
12239,stenrev,81.0,1.0,0.000445,3.691390e-07,1206.485592,0.000829,stenrive


In [ ]:
unique_words_lemma[unique_words_lemma.word=="stenrive"]

,word,word_count_C,word_count_NC,frequency_C,frequency_NC,odds_C,odds_NC
9605,stenrive,9.0,1.0,0.000049,3.691390e-07,134.053955,0.00746


In [ ]:
count = 0
for i in range(len(unique_words_C_list)):
  if "stenrev" == unique_words_C_list[i]:
    count += 1
    if "stenrive" != unique_words_lemma_C_list[i]:
      print(unique_words_lemma_C_list[i])

print(count)

stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenreve
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
stenrev
81
